In [ ]:
#@title Загружаем билиотеки
import os
import shutil
from shutil import copytree
import zipfile
import cv2
import albumentations as A
import numpy as np
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import uuid
from PIL import Image
import re
import random
#from pyunpack import Archive

from google.colab import drive, files                                           # подключем диск, # Загрузка файлов                                                                  # импортируем модуль для удобной работы с файлами
import gdown
print(f"\x1b[34mБиблиотеки загружены.")

Библиотеки загружены.


In [ ]:
#@title Монтируем диск
try:
    drive.mount('/content/drive')
    print(f"\x1b[34mGoogle Drive успешно подключен.")
except Exception as e:
    print("Не удалось подключить Google Drive:", e)

Mounted at /content/drive
Google Drive успешно подключен.


In [ ]:
#@title Скачиваем файл данных - начал работать
path_file = 'https://drive.google.com/uc?id=1zhD07nb1Ycz_T4NG0zkKgnW1J6u5-foB' # Karate mixed 11 classes 562 Images _ crop X-X.zip
gdown.download(path_file, None, quiet = True)
!echo "Скачивание архива успешно завершено!"

Скачивание архива успешно завершено!


In [ ]:
#@title Запрос файла с помощью request - пока непонятно как работает
import requests

file_id = '1MSxHthFy4I8j-nCXdFfuyvWpdchE-4X2'
url = f'https://drive.google.com/uc?id={file_id}'

response = requests.get(url)
if response.status_code == 200:
    with open('downloaded_file.txt', 'wb') as file:
        file.write(response.content)
    print('Файл успешно скачан.')
else:
    print(f'Не удалось скачать файл. Статус код: {response.status_code}')

In [ ]:
#@title Копируем файл архива - резервный способ
# Укажите путь для архива

zip_file_path = '/content/drive/MyDrive/Colab Notebooks/Datas/Zips/Karate mixed 11 classes New Data Origin Edited 1000.zip'
send_zip = '/content/Karate mixed 11 classes New Data Origin Edited 1000.zip'
# Копирование
shutil.copy(zip_file_path, send_zip)

'/content/Karate mixed 11 classes New Data Origin Edited 1000.zip'

In [ ]:
#@title Разархивируем архив данных - странно не все файлы распаковывает
!unzip -q "Karate_mixed_11_classes_crop_224x224_aug.zip"
!echo "Распаковка архива успешно завершена!"

Распаковка архива успешно завершена!


In [ ]:
#@title Разархивируем архив данных программно
def unzip_archive(zip_path, extract_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)

# Пример использования
zip_file_path = "/content/Karate mixed 11 classes New Data Origin Edited 1000.zip"
extract_folder_path = "/content/"

# Создаем папку для извлеченных файлов, если её нет
#os.makedirs(extract_folder_path, exist_ok=True)

# Разархивируем архив
unzip_archive(zip_file_path, extract_folder_path)

In [ ]:
#@title Переименование файлов для обучения модели
def extract_prefix(folder_name):
    # Используем регулярное выражение для извлечения текста в скобках
    match = re.search(r'\((.*?)\)', folder_name)
    return match.group(1) if match else folder_name

def rename_files(folder_path):
    subfolders = [f.path for f in os.scandir(folder_path) if f.is_dir()]

    for subfolder in subfolders:
        # Получаем префикс из имени подпапки
        prefix = extract_prefix(os.path.basename(subfolder))

        # Получаем список файлов в текущей подпапке
        files = os.listdir(subfolder)

        # Перемешиваем список файлов
        random.shuffle(files)

        i = 1  # Начальное значение для номера файла

        for filename in files:
            old_filepath = os.path.join(subfolder, filename)

            unique_prefix = prefix.lower().replace(" ", "_")
            new_filename = f"{unique_prefix}_{i}.jpg"
            new_filepath = os.path.join(subfolder, new_filename)

            os.rename(old_filepath, new_filepath)
            print(f"Файл {filename} переименован в {new_filename}")

            i += 1  # Инкрементируем номер файла

# Переименование файлов в обучающей выборке
folder_path_train = "/content/Karate mixed 11 classes 562 Images _ crop X-X/Karate train_224x224 aug"
rename_files(folder_path_train)

# Переименование файлов в тестовой выборке
folder_path_test = "/content/Karate mixed 11 classes 562 Images _ crop X-X/Karate test_224x224 aug"
rename_files(folder_path_test)

Выходные данные были обрезаны до нескольких последних строк (5000).
Файл zenkutsu_dachi_121_1.jpg переименован в zenkutsu_dachi_2061.jpg
Файл zenkutsu_dachi_94_4.jpg переименован в zenkutsu_dachi_2062.jpg
Файл zenkutsu_dachi_218_0.jpg переименован в zenkutsu_dachi_2063.jpg
Файл zenkutsu_dachi_404_4.jpg переименован в zenkutsu_dachi_2064.jpg
Файл zenkutsu_dachi_183_2.jpg переименован в zenkutsu_dachi_2065.jpg
Файл zenkutsu_dachi_316_1.jpg переименован в zenkutsu_dachi_2066.jpg
Файл zenkutsu_dachi_176_4.jpg переименован в zenkutsu_dachi_2067.jpg
Файл zenkutsu_dachi_90_1.jpg переименован в zenkutsu_dachi_2068.jpg
Файл zenkutsu_dachi_428_4.jpg переименован в zenkutsu_dachi_2069.jpg
Файл zenkutsu_dachi_34_2.jpg переименован в zenkutsu_dachi_2070.jpg
Файл zenkutsu_dachi_139_2.jpg переименован в zenkutsu_dachi_2071.jpg
Файл zenkutsu_dachi_409_3.jpg переименован в zenkutsu_dachi_2072.jpg
Файл zenkutsu_dachi_199_1.jpg переименован в zenkutsu_dachi_2073.jpg
Файл zenkutsu_dachi_192_4.jpg переимен

In [ ]:
#@title Архивация данных и копирование в целевую папку
# Архивируем папку с аугментированными изображениями
def create_zip_archive(zip_file_path, source_dirs):
    # Создание архива с аугментированными изображениями
    with zipfile.ZipFile(zip_file_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for source_dir in tqdm(source_dirs, desc = "Общий прогресс"):
            for dir_path, _, file_names in os.walk(source_dir):
                for file_name in tqdm(file_names, desc = f"Архивирование: {dir_path}"):
                    file_path = os.path.join(source_dir, dir_path, file_name)

                    arcname = os.path.relpath(file_path, start = "/content/")

                    zipf.write(file_path, arcname = arcname)

    # Укажите путь для архива
    send_zip = '/content/drive/MyDrive/Colab Notebooks/Datas/Zips/Karate_Dataset_aug_shufle.zip'
    # Копирование
    shutil.copy(zip_file_path, send_zip)

    print("Архивирование и копирование завершены.")

# Список папок, которые вы хотите архивировать
source_dirs = ['/content/Karate mixed 11 classes 562 Images _ crop X-X/Karate train_224x224 aug',
               '/content/Karate mixed 11 classes 562 Images _ crop X-X/Karate test_224x224 aug']

# Используйте функцию для архивирования
create_zip_archive('/content/Karate_Dataset_aug_shufle.zip', source_dirs)

Общий прогресс:   0%|          | 0/2 [00:00<?, ?it/s]

Архивирование: /content/Karate mixed 11 classes 562 Images _ crop X-X/Karate train_224x224 aug: 0it [00:00, ?i…

Архивирование: /content/Karate mixed 11 classes 562 Images _ crop X-X/Karate train_224x224 aug/Хэйсоку дачи (H…

Архивирование: /content/Karate mixed 11 classes 562 Images _ crop X-X/Karate train_224x224 aug/Санчин дачи (Sa…

Архивирование: /content/Karate mixed 11 classes 562 Images _ crop X-X/Karate train_224x224 aug/Шико Дачи (Shik…

...

Архивирование: /content/Karate mixed 11 classes 562 Images _ crop X-X/Karate test_224x224 aug/Хачиджи Дачи (So…

Архивирование: /content/Karate mixed 11 classes 562 Images _ crop X-X/Karate test_224x224 aug/Дзенкуцу Дачи (Z…

Архивирование: /content/Karate mixed 11 classes 562 Images _ crop X-X/Karate test_224x224 aug/Киба Дачи (Kiba …

Архивирование: /content/Karate mixed 11 classes 562 Images _ crop X-X/Karate test_224x224 aug/Мусуби дачи (Mus…

Архивирование и копирование завершены.
